In [9]:
#import atoti as tt
session = tt.Session.start()
sales_table = session.read_csv("data/sales.csv", keys={"Sale ID"}, table_name="Sales")
sales_table.head()
cube = session.create_cube(sales_table)

# Aliasing the hierarchies property to a shorter variable name because we will use it a lot.
h = cube.hierarchies

l = cube.levels
m = cube.measures

cube.query(m["Quantity.SUM"])
cube.query(m["Quantity.SUM"], levels=[l["Shop"]])
cube.query(
    m["Quantity.SUM"],
    filter=l["Shop"] == "shop_0",
)
cube.query(m["Quantity.SUM"], levels=[l["Date"], l["Product"]])

cube.query(
    m["Quantity.SUM"],
    levels=[l["Date"], l["Product"]],
    filter=l["Shop"] == "shop_0",
)
session.widget
session.widget
session.widget
session.widget
session.link

products_table = session.read_csv(
    "data/products.csv", keys={"Product"}, table_name="Products"
)
products_table.head()
sales_table.join(products_table, sales_table["Product"] == products_table["Product"])
session.tables.schema
h
session.widget
session.widget
m["Purchase price.VALUE"] = tt.agg.single_value(products_table["Purchase price"])
cube.query(m["Purchase price.VALUE"], levels=[l["Product"]])
shops_table = session.read_csv("data/shops.csv", keys={"Shop ID"}, table_name="Shops")
shops_table.head()
sales_table.join(shops_table, sales_table["Shop"] == shops_table["Shop ID"])
session.tables.schema
m["Max price"] = tt.agg.max(sales_table["Unit price"])
cube.query(m["Max price"], include_totals=True, levels=[l["Category"]])
m["Amount.SUM"] = tt.agg.sum(sales_table["Quantity"] * sales_table["Unit price"])
m["Amount.MEAN"] = tt.agg.mean(
    sales_table["Quantity"] * sales_table["Unit price"],
)
session.widget
cost = tt.agg.sum(
    m["Quantity.SUM"] * tt.agg.single_value(products_table["Purchase price"]),
    scope=tt.OriginScope({l["Product"]}),
)
m["Margin"] = m["Amount.SUM"] - cost
m["Margin rate"] = m["Margin"] / m["Amount.SUM"]
cube.query(m["Margin"], m["Margin rate"], levels=[l["Product"]])
session.widget
m["Cumulative amount"] = tt.agg.sum(
    m["Amount.SUM"],
    scope=tt.CumulativeScope(l["Date"]),
)
session.widget
m["Average amount per shop"] = tt.agg.mean(
    m["Amount.SUM"],
    scope=tt.OriginScope({l["Shop"]}),
)
cube.query(
    m["Average amount per shop"],
    include_totals=True,
    levels=[l["Sub category"]],
)
h["Product"] = [l["Category"], l["Sub category"], l["Product"]]
del h["Category"]
del h["Sub category"]
h["Geography"] = [
    shops_table["Country"],
    shops_table["State or region"],
    shops_table["City"],
]
del h["Country"]
del h["State or region"]
del h["City"]
h["Geography"].dimension = "Location"
h
m["Parent category amount"] = tt.parent_value(
    m["Amount.SUM"],
    degrees={h["Products", "Product"]: 1},
)
m["Percent of parent amount"] = m["Amount.SUM"] / m["Parent category amount"]
session.widget
del m["Unit price.SUM"]
m["Parent category amount"].visible = False
for measure in [
    m["Amount.MEAN"],
    m["Amount.SUM"],
    m["Average amount per shop"],
    m["Cumulative amount"],
    m["Percent of parent amount"],
]:
    measure.folder = "Amount"
    cube.query(m["Percent of parent amount"], m["Margin rate"], levels=[l["Category"]])
    m["Percent of parent amount"].formatter = "DOUBLE[0.00%]"
m["Margin rate"].formatter = "DOUBLE[0.00%]"
cube.query(m["Percent of parent amount"], m["Margin rate"], levels=[l["Category"]])

import pandas as pd
products_df = pd.read_csv("data/products.csv")
products_df.head()
better_prices = {
    "TAB_0": 180.0,
    "TAB_1": 250.0,
    "CHA_2": 40.0,
    "BED_3": 110.0,
    "BED_4": 210.0,
}
for product, purchase_price in better_prices.items():
    products_df.loc[products_df["Product"] == product, "Purchase price"] = (
        purchase_price
    )
products_df.head()
products_table.scenarios["Cheaper purchase prices"].load(products_df)
session.widget
session.widget
country_simulation = cube.create_parameter_simulation(
    "Country Simulation",
    levels=[l["Country"]],
    measures={"Country parameter": 1.0},
)

country_simulation += ("France Crisis", "France", 0.80)
country_simulation.head()
m["Quantity.SUM"] = tt.agg.sum(
    tt.agg.sum(sales_table["Quantity"]) * m["Country parameter"],
    scope=tt.OriginScope({l["Country"]}),
)
m["Amount.SUM"] = tt.agg.sum(
    tt.agg.sum(sales_table["Unit price"] * sales_table["Quantity"])
    * m["Country parameter"],
    scope=tt.OriginScope({l["Country"]}),
)
cube.query(
    m["Quantity.SUM"],
    m["Amount.SUM"],
    include_totals=True,
    levels=[l["Country Simulation"], l["Country"]],
)
cube.query(m["Cumulative amount"], levels=[l["Country Simulation"], l["Country"]])
country_simulation += ("US boost", "USA", 1.15)
cube.query(m["Quantity.SUM"], levels=[l["Country Simulation"], l["Country"]])
session.widget
session.widget

RuntimeError: Failed to extract Java version from
Property settings:
    awt.toolkit = sun.lwawt.macosx.LWCToolkit
    file.encoding = UTF-8
    file.encoding.pkg = sun.io
    file.separator = /
    ftp.nonProxyHosts = local|*.local|169.254/16|*.169.254/16
    gopherProxySet = false
    http.nonProxyHosts = local|*.local|169.254/16|*.169.254/16
    java.awt.graphicsenv = sun.awt.CGraphicsEnvironment
    java.awt.printerjob = sun.lwawt.macosx.CPrinterJob
    java.class.path = .
    java.class.version = 52.0
    java.endorsed.dirs = /Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home/lib/endorsed
    java.ext.dirs = /Users/hamzaoujja/Library/Java/Extensions
        /Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home/lib/ext
        /Library/Java/Extensions
        /Network/Library/Java/Extensions
        /System/Library/Java/Extensions
        /usr/lib/java
    java.home = /Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home
    java.io.tmpdir = /var